In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PRK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [9]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [10]:
list(zip(longforms, counts))

[('photorefractive keratectomy', 107),
 ('phosphoribulokinase', 76),
 ('pranlukast', 8),
 ('ribulose 5 phosphate kinase', 3),
 ('prk', 2),
 ('protein kinase c related kinase', 2),
 ('primary rabbit kidney', 1),
 ('pkc related kinase', 1),
 ('plant receptor kinase', 1),
 ('protein kinase c related protein', 1),
 ('performing excimer laser surface ablation', 1)]

In [13]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-29 04:51:23] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [14]:
result = [grounding_map, names, pos_labels]

In [15]:
result

[{'performing excimer laser surface ablation': 'ungrounded',
  'phosphoribulokinase': 'UP:P25697',
  'photorefractive keratectomy': 'MESH:D018903',
  'pkc related kinase': 'FPLX:PKN',
  'plant receptor kinase': 'ungrounded',
  'pranlukast': 'CHEBI:CHEBI:94810',
  'primary rabbit kidney': 'ungrounded',
  'prk': 'ungrounded',
  'protein kinase c related kinase': 'FPLX:PKN',
  'protein kinase c related protein': 'FPLX:PKN',
  'ribulose 5 phosphate kinase': 'UP:P25697'},
 {'UP:P25697': 'Phosphoribulokinase, chloroplastic',
  'MESH:D018903': 'Photorefractive Keratectomy',
  'FPLX:PKN': 'PKN',
  'CHEBI:CHEBI:94810': 'N-[4-oxo-2-(2H-tetrazol-5-yl)-1-benzopyran-8-yl]-4-(4-phenylbutoxy)benzamide'},
 ['CHEBI:CHEBI:94810', 'FPLX:PKN', 'MESH:D018903', 'UP:P25697']]

In [47]:
grounding_map, names, pos_labels = [{'performing excimer laser surface ablation': 'ungrounded',
  'phosphoribulokinase': 'UP:P25697',
  'photorefractive keratectomy': 'MESH:D018903',
  'pkc related kinase': 'FPLX:PKN',
  'plant receptor kinase': 'ungrounded',
  'pranlukast': 'CHEBI:CHEBI:94810',
  'primary rabbit kidney': 'ungrounded',
  'prk': 'ungrounded',
  'protein kinase c related kinase': 'FPLX:PKN',
  'protein kinase c related protein': 'FPLX:PKN',
  'ribulose 5 phosphate kinase': 'UP:P25697'},
 {'UP:P25697': 'Phosphoribulokinase, chloroplastic',
  'MESH:D018903': 'Photorefractive Keratectomy',
  'FPLX:PKN': 'PKN',
  'CHEBI:CHEBI:94810': 'N-[4-oxo-2-(2H-tetrazol-5-yl)-1-benzopyran-8-yl]-4-(4-phenylbutoxy)benzamide'},
 ['CHEBI:CHEBI:94810', 'FPLX:PKN', 'MESH:D018903', 'UP:P25697']]

In [48]:
excluded_longforms = ['prk']

In [49]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [50]:
additional_entities = {}

In [51]:
unambiguous_agent_texts = {}

In [52]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [53]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [54]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [55]:
intersection1

[]

In [56]:
intersection2

[]

In [57]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [58]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [59]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 04:54:31] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 04:54:32] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9368775005827068 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [60]:
classifier.stats

{'label_distribution': {'UP:P25697': 61,
  'ungrounded': 3,
  'MESH:D018903': 81,
  'FPLX:PKN': 4,
  'CHEBI:CHEBI:94810': 1},
 'f1': {'mean': 0.936878, 'std': 0.006874},
 'precision': {'mean': 0.917214, 'std': 0.014176},
 'recall': {'mean': 0.95931, 'std': 0.012996},
 'FPLX:PKN': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D018903': {'f1': {'mean': 0.976471, 'std': 0.028818},
  'pr': {'mean': 0.988235, 'std': 0.023529},
  'rc': {'mean': 0.966013, 'std': 0.044789}},
 'UP:P25697': {'f1': {'mean': 0.961823, 'std': 0.034406},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.928571, 'std': 0.063888}},
 'CHEBI:CHEBI:94810': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}}}

In [61]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [62]:
disamb.dump(model_name, results_path)

In [63]:
print(disamb.info())

Disambiguation model for PRK

Produces the disambiguations:
	N-[4-oxo-2-(2H-tetrazol-5-yl)-1-benzopyran-8-yl]-4-(4-phenylbutoxy)benzamide*	CHEBI:CHEBI:94810
	PKN*	FPLX:PKN
	Phosphoribulokinase, chloroplastic*	UP:P25697
	Photorefractive Keratectomy*	MESH:D018903

Class level metrics:
--------------------
Grounding                                                                   	Count	F1     
                                                 Photorefractive Keratectomy*	81	0.97647
                                          Phosphoribulokinase, chloroplastic*	61	0.96182
                                                                         PKN*	 4	    0.0
                                                                  Ungrounded	 3	    0.0
N-[4-oxo-2-(2H-tetrazol-5-yl)-1-benzopyran-8-yl]-4-(4-phenylbutoxy)benzamide*	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.93688
	Precision:	0.91721
	Recall:		0.95931

* Positive labels
See Docstring for explanation



In [64]:
model_to_s3(disamb)